## Commons helper: uploads from lamoncloa.gob.es

This notebook helps users to upload images from the [photographic galleries](http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/Paginas/index.aspx) at `lamoncloa.gob.es`.

In this case, I have found problems when running the notebook from [PAWS](http://paws.wmflabs.org/), as there seems to be antiscraping features implemented in La Moncloa web site. Thus, run this notebook from your own environment.

#### Prerequisites:
- Create a Python 3 virtual environment and activate it.
- Install `pywikibot`:
```bash
pip install pywikibot
```
- Install `mako`:
```bash
pip install mako
```
or:
```bash
conda install -c anaconda mako
```
- Install `dateparser`:
```bash
pip install dateparser
```
or:
```bash
 conda install -c conda-forge dateparser 
```
- Create a properly formatted `user-config.py` file.
- Launch `jupyter notebook`

#### Configuration
This notebook takes all the photograms in a given URL (provided that this URL hosts a photographic gallery) and uploads them to commons inserting the proper license templates. The following features are automatically extracted:
- **Image name**: The mane of the images is taken from the title of the photographic gallery. An autoincremental number is appended to distinguish between all the photographs.
- **Image description**: The description is copied from the image description at the photographic gallery.
- **Image date**: The date is extracted from the image description.
- **Phothographer**: The information is extracted from the image description.
- **Categories**: Some categories are automaticaly extracted from the image description. For instance, the notebook assigns `Palacio de la Moncloa` and `[month] [year] in Madrid` if the photograph has been taken at the seat of the government, or `Photographs by [photographer]` if available at the image description.

**However** you can override or update most of them by editing the `config` dictionary in the notebook, add additional categories or determine which images to upload:
- `url`: This is where the photographic gallery is available. This configuration element is **mandatory**.
- `categories`: Include here as many categories as you want to assign to all images (for a category for a particular image you must do it afterwards). If empty, no categories but the automatically detected will be added.
- `uploader_category`: If you wish to assign a category for you as uploader, do it here. If empty, no category will be added.
- `title`: Include your own name if you don't like the one being extracted.
- `pub_date`: Use the following format: YYYY-MM-DD (i.e. 2018-13-24)
- `excluded`: A list with the indices of the pictures you don't wish to upload

In [ ]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import pywikibot as pb
from pywikibot.specialbots import UploadRobot

import requests
from requests.compat import quote
from bs4 import BeautifulSoup
from mako.template import Template
from io import BytesIO

import dateparser as dp
import re
import shutil
from hashlib import sha1
import os

commons_site = pb.Site("commons", "commons")

In [ ]:
def is_commons_file (sha):
    """Determine whether the image is already in Commons"""
    result = False
    for page in pb.site.APISite.allimages(commons_site, sha1=sha) :
        print (page)
        result = True
        break
    return result
    
def get_hash (file_path):
    """Get the SHA-1 hash of an image"""
    image_file = open(file_path, "rb")
    byte_stream = BytesIO(image_file.read()).getvalue()
    image_file.close()
    
    hashObject = sha1()
    hashObject.update(byte_stream)
    return hashObject.hexdigest()

def remove_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
# Creation of images folder
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [ ]:
# Configuration
config = {
    'url': 'http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/presidente/Paginas/2018/210518rajoy_vigo.aspx',
    'categories': ['Mariano Rajoy in 2018',
                   '2018 in Vigo',
                   'Alberto Núñez Feijoo'
                  ],
    'uploader_category': None,
    'title': None,
    'pub_date': None,
    'excluded': []
}
categories = [category for category in (config['categories'] + [config['uploader_category']]) if category]

In [ ]:
# Retrieval of base page for extracting gallery information
while True:
    request = requests.get(config['url'])
    if request is not None:
        #print("{0}".format(u''.join(request.text).encode('utf-8')))
        response = request.text
        break
    else :
        print ('Trying')

soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Gallery title
if not config['title']:
    title = soup.find_all("h1", class_="contenidoTitulo")[0].get_text().strip().replace(':', ' - ').replace('  ', ' ')
else:
    title = config['title']
title

In [ ]:
# Image date
if not config['pub_date']:
    try:
        date_string = soup.find_all("div", class_="contenidoFecha")[0].get_text()
        pub_date_sp = ' '.join(date_string.strip().split(' ')[-5:])
        #print (pub_date_sp)
        parsed_pub_date = dp.parse(pub_date_sp)
        pub_date = parsed_pub_date.strftime('%Y-%m-%d')
    
        year_date = parsed_pub_date.strftime('%Y')
        month_date = parsed_pub_date.strftime('%B')
        if 'La Moncloa' in date_string:
            categories.append('Palacio de la Moncloa')
            categories.append(f'{month_date} {year_date} in Madrid')
    except :
        pub_date = None
else:
    pub_date = config['pub_date']
    
pub_date

In [ ]:
# Image list extraction
p = re.compile("gFIO\[\d{1,2}\] = \\'(.*)\\';")
image_list = ['http://www.lamoncloa.gob.es' + url for url in re.findall(p, response)]
image_list

In [ ]:
# Image description extraction
p = re.compile("gFT\[\d{1,2}\] = \\'(.*)\\';")
description_list = [remove_tags(desc.strip().rstrip('.')) for desc in re.findall(p, response)]
description_list

In [ ]:
# Image author extraction
p = re.compile("gFA\[\d{1,2}\] = \\'(.*)\\';")
author_list = [author.strip() for author in re.findall(p, response)]
author_list

In [ ]:
truth_list = ['Pool' in author for author in author_list]
truth_list

In [ ]:
# Page of the image in Commons
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Galería fotográfica: ${title}]
|author=Ministerio de la Presidencia. Gobierno de España (${author})
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

${cat_string}"""

vars = {
    "url": config['url'],
    "description": None,
    "author": None,
    "date": pub_date,
    "title": title,
    "cat_string": None
}

In [ ]:
# Image retrieval and upload to Commons
excluded = config['excluded']

used_names = []
global_counter = 1
for i, image in enumerate(image_list):
    # If the image is excluded, skip
    if i in excluded:
        print ("Image excluded. Skipping")
        continue
    elif truth_list[i] == False:
        print ("Image from a party other than Moncloa. Skipping")
        continue        
        
    # First, the image is downloaded and stored
    image_url = quote(image.encode('utf-8'), ':/')
    print(image_url)
    image_name = '{} {:02d}.jpg'.format(title.replace(':', ' -').replace('  ', ' '), global_counter)
    global_counter += 1
    image_path = os.path.join(images_directory, image_name)
    while True:
        print ('Trying download')
        r = requests.get(image_url, stream=True)
        if r is not None:
            with open(image_path, 'wb') as out_file:
                shutil.copyfileobj(r.raw, out_file)
            print ('Image downloaded. Starting upload process')
            break
        else :
            print ('Still trying download')

    # If the image is already in Commons, it is skipped
    if is_commons_file(get_hash(image_path)) :
        print ("Image already in commons. Skipping")
        os.remove(image_path)
        global_counter -= 1
        continue

    # If the image name is already in commons, a new name is found
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)
        
    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # Image upload
    vars["description"] = description_list[i]
    vars["author"] = author_list[i]
    if 'Sendra' in author_list[i]:
        categories.append('Photographs by César P. Sendra')
    elif 'Cuadrado' in author_list[i]:
        categories.append('Photographs by J. M. Cuadrado')
    elif 'Crespo' in author_list[i]:
        categories.append('Photographs by Diego Crespo')
    categories = list(set(categories))
    print (categories)
    vars["cat_string"] = '\n'.join(['[[Category:'+i+']]' for i in categories])
    t = Template(template)
    _text = t.render(**vars)

    bot = UploadRobot([image_path],
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)